In [1]:
import os
os.chdir("/content/drive/MyDrive/DREAM")
!ls

 Data				      'Eval trials.gsheet'   hparams.ipynb
 dictionary_73_11.pt		       examine.ipynb	     new_dictionary.pt
 dictionary.pt			       finetune.ipynb	     SECOND
 dream				       FIRST		     submit.ipynb
'dream (1)'			       Forms		    'Submit results.gdoc'
'Dream Olfactory Mixtures Plan.gdoc'   FOURTH		     THIRD
'Dream Writeup.gdoc'		       ft_examine.ipynb      Trials
 eval.ipynb			       FT_FIRST		     vectorizing.ipynb


In [2]:
!pip install rdkit
!pip install ogb
!pip install torch-geometric
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.w

In [3]:
!rm -rf dream
!git clone https://github.com/laurahsisson/dream.git
import sys
sys.path.append("dream")

Cloning into 'dream'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 250 (delta 0), reused 0 (delta 0), pack-reused 247
Receiving objects: 100% (250/250), 958.89 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [4]:
import json
with open("Data/odor_pair_266.json") as f:
  dataset = json.load(f)

dataset = [d for d in dataset if d["mol1"] and d["mol2"]]

In [5]:
all_smiles = set()
for d in dataset:
  all_smiles.add(d["mol1"])
  all_smiles.add(d["mol2"])
len(all_smiles),next(iter(all_smiles))

(3428, 'CCCCCCCC(OCC)OCC')

In [6]:
import torch
from ogb.utils import smiles2graph
from torch_geometric.data import InMemoryDataset, download_url, Data
import tqdm
import networkx as nx

def to_torch(smiles):
  graph = smiles2graph(smiles)
  tensor_keys = ["edge_index", 'edge_feat', 'node_feat']
  for key in tensor_keys:
      graph[key] = torch.tensor(graph[key])
  return graph

graph_data = []
for smiles in tqdm.tqdm(all_smiles):
  try:
    graph = to_torch(smiles)
    gd = Data(x=graph["node_feat"].float(),edge_attr=graph["edge_feat"],edge_index=graph["edge_index"],smiles=smiles)
    graph_data.append(gd)
  except AttributeError as e:
    continue
len(graph_data), graph_data[0]

 17%|█▋        | 581/3428 [00:00<00:02, 1186.44it/s][17:38:25] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[17:38:25] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
 69%|██████▉   | 2380/3428 [00:01<00:00, 1756.90it/s][17:38:26] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[17:38:26] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'
100%|██████████| 3428/3428 [00:02<00:00, 1532.02it/s]


(3425,
 Data(x=[14, 9], edge_index=[2, 26], edge_attr=[26, 3], smiles='CCCCCCCC(OCC)OCC'))

In [7]:
odor_x = torch.concat([gd.x for gd in graph_data],dim=0)
unique_x = torch.unique(odor_x,dim=0)
odor_edge = torch.concat([gd.edge_attr for gd in graph_data],dim=0)
unique_edge_attr = torch.unique(odor_edge,dim=0)

unique_x.shape, unique_edge_attr.shape

(torch.Size([73, 9]), torch.Size([11, 3]))

In [8]:
odor_x_set = {tuple(x.numpy()) for x in odor_x}
odor_edge_set = {tuple(x.numpy()) for x in odor_edge}
len(odor_x_set), len(odor_edge_set)

(73, 11)

In [9]:
from torch_geometric.data import InMemoryDataset, download_url, Data, Batch

class PairData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_s':
            return self.x_s.size(0)
        if key == 'edge_index_t':
            return self.x_t.size(0)
        return super().__inc__(key, value, *args, **kwargs)

In [10]:
import h5py
import tqdm
import numpy as np
import torch
import dream.dreamdata

train_data = dream.dreamdata.load_dream_h5('Data/train.h5')
train_data[0]

100%|██████████| 500/500 [00:02<00:00, 190.40it/s]


{'idx': 0,
 'label': 'Snitz 1_1+Snitz 1_2',
 'mixture1': '1.0',
 'mixture2': '2.0',
 'graph1': BlendData(x=[97, 9], edge_index=[2, 190], edge_attr=[190, 3], blend_batch=[10], mol_batch=[97]),
 'graph2': BlendData(x=[96, 9], edge_index=[2, 192], edge_attr=[192, 3], blend_batch=[10], mol_batch=[96]),
 'dataset': 'Snitz 1',
 'y': tensor(0.6042)}

In [11]:
test_data = dream.dreamdata.load_dream_h5("Data/test.h5")
leaderboard_data = dream.dreamdata.load_dream_h5("Data/leaderboard.h5")

100%|██████████| 46/46 [00:00<00:00, 187.86it/s]


In [12]:
train_x = torch.concat([gd["graph1"].x for gd in train_data]+[gd["graph2"].x for gd in train_data],dim=0)
test_x = torch.concat([gd["graph1"].x for gd in test_data]+[gd["graph2"].x for gd in test_data],dim=0)
leaderboard_x = torch.concat([gd["graph1"].x for gd in leaderboard_data]+[gd["graph2"].x for gd in leaderboard_data],dim=0)
all_dream_x = torch.concat([train_x,test_x,leaderboard_x],axis=0)
display(all_dream_x.shape)
dream_unique_x = {tuple(x.numpy()) for x in all_dream_x}
len(dream_unique_x)

torch.Size([180742, 9])

34

In [13]:
for x in dream_unique_x:
  if not x in odor_x_set:
    print(x)

In [14]:
train_edge = torch.concat([gd["graph1"].edge_attr for gd in train_data]+[gd["graph2"].edge_attr for gd in train_data],dim=0)
test_edge = torch.concat([gd["graph1"].edge_attr for gd in test_data]+[gd["graph2"].edge_attr for gd in test_data],dim=0)
leaderboard_edge = torch.concat([gd["graph1"].edge_attr for gd in leaderboard_data]+[gd["graph2"].edge_attr for gd in leaderboard_data],dim=0)
all_dream_edge = torch.concat([train_edge,test_edge,leaderboard_edge],axis=0)
display(all_dream_edge.shape)
dream_unique_edge = {tuple(x.numpy()) for x in all_dream_edge}
len(dream_unique_edge)

torch.Size([344546, 3])

6

In [15]:
for x in dream_unique_edge:
  if not x in odor_edge_set:
    print(x)

In [16]:
all_x = dream_unique_x.union(odor_x_set)
all_edge = dream_unique_edge.union(odor_edge_set)
len(all_x), len(all_edge)

(73, 11)

In [17]:
dictionary = {"x":list(all_x),"edge_attr":list(all_edge)}
torch.save(dictionary,"dictionary.pt")